### Imports

In [1]:
from IPython.display import clear_output

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q path.py
!pip install -q pytorch3d
# https://github.com/facebookresearch/pifuhd/issues/77
!pip install -q 'torch==1.6.0+cu101' -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q 'torchvision==0.7.0+cu101' -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q 'pytorch3d==0.2.5'
!pip install -q Ninja
clear_output()

In [4]:
import numpy as np
import math
import random
import os
import torch
import scipy.spatial.distance
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, utils
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pytorch3d

import plotly.graph_objects as go
import plotly.express as px

from path import Path

from pytorch3d.loss import chamfer

random.seed = 42

In [5]:
!wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip # /ModelNet40.zip - 40 classes
!unzip -q ModelNet10.zip

path = Path("ModelNet10")

folders = [dir for dir in sorted(os.listdir(path)) if os.path.isdir(path/dir)]

clear_output()
classes = {folder: i for i, folder in enumerate(folders)}
# classes

#### Imports from helping.py

In [6]:
!gdown https://drive.google.com/uc?id=1CVwVxdfUfP6TRcVUjjJvQeRcgCGcnSO_
from helping import *
clear_output()

### Load Data


In [7]:
trainloader_pre = torch.load('drive/MyDrive/Thesis/dataloaders/final/dataloader_beds_pre/trainloader.pth')
validloader_pre = torch.load('drive/MyDrive/Thesis/dataloaders/final/dataloader_beds_pre/validloader.pth')

trainloader_both = torch.load('drive/MyDrive/Thesis/dataloaders/final/dataloader_beds_both/trainloader.pth')
validloader_both = torch.load('drive/MyDrive/Thesis/dataloaders/final/dataloader_beds_both/validloader.pth')

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Autoencoders

### PCAutoEncoder

In [9]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.1)

In [10]:
from torch.utils.tensorboard import SummaryWriter

from enum import Enum
class DecoderType(Enum):
    ORIGINAL = 1
    INCREASE_POINTS = 2
    INCREASE_CHANNELS = 3
    
class DataType(Enum):
    AUG_PRE = 1 # augmentation during training
    AUG_DUR = 2
    AUG_BOTH = 3

In [11]:
class PointNetAE(nn.Module):
    def __init__(self, num_points=1024, z_dim=100, decoder_type=DecoderType.ORIGINAL):
        super(PointNetAE, self).__init__()
        self.num_points = num_points

        self.encoder = PointEncoder(num_points, z_dim=z_dim)
        if decoder_type is DecoderType.INCREASE_POINTS:
            self.decoder = PointDecoderPoints(num_points, z_dim=z_dim)
        elif decoder_type is DecoderType.INCREASE_CHANNELS:
            self.decoder = PointDecoderChannels(num_points, z_dim=z_dim)
        else:
            self.decoder = PointDecoderOriginal(num_points, z_dim=z_dim)

        self.name = self.decoder.name

    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var / 2)
        eps = torch.randn_like(std)
        return mu + std * eps

    def forward(self, x):
        x, mu, logvar = self.encoder(x)
        # x = self.reparameterize(mu, logvar)
        x = self.decoder(x)
        return x


class PointEncoder(nn.Module):
    def __init__(self, num_points, z_dim):
        super(PointEncoder, self).__init__()
        self.num_points = num_points
        self.feature_dim = z_dim
        self.convs = nn.Sequential(
            nn.Conv1d(3, 64, 1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, 128, 1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, num_points, 1),
            nn.BatchNorm1d(num_points),
        )

        self.dense = nn.Sequential(
            nn.Linear(num_points, 512),
            nn.ReLU(),
            nn.Linear(512, self.feature_dim)
        )

        self.dense.apply(init_weights)

        self.mu_fc = nn.Linear(self.feature_dim, z_dim)
        self.log_var_fc = nn.Linear(self.feature_dim, z_dim)

    def forward(self, x):
        x = self.convs(x)
        x, _ = torch.max(x, 2) # instead of maxpool
        x = x.view(-1, self.num_points)
        x = self.dense(x)
        x_relu = torch.relu(x)
        mu, log_var = self.mu_fc(x_relu), self.log_var_fc(x_relu)
        return x, mu, log_var


'''
use only dense layers in decoder
dec1_aug1_1024_48.pt
dec1_aug3_1024_48.pt
'''
class PointDecoderOriginal(nn.Module):
    def __init__(self, num_points, z_dim):
        super(PointDecoderOriginal, self).__init__()
        self.name = f'model_dense'
        self.num_points = num_points
        self.dense_layers = nn.Sequential(
            nn.Linear(z_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, num_points),
            nn.Linear(num_points, num_points*3),
            nn.Tanh()
        )
        self.dense_layers.apply(init_weights)

    def forward(self, x):
        batchsize = x.size()[0]
        x = self.dense_layers(x)
        x = x.view(batchsize, 3, self.num_points)
        return x

'''
apply Conv1d to increase dimensionality (1 -> 3), 3: x, y, z
'''
class PointDecoderChannels(nn.Module):
    def __init__(self, num_points, z_dim):
        super(PointDecoderChannels, self).__init__()
        self.num_points = num_points
        self.name = 'model_conv1d_channels'
        self.dense_layers = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.Dropout(0.05),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.Dropout(0.05),
            nn.ReLU(),
            nn.Linear(512, num_points),
            nn.ReLU(),
        )
        self.conv = nn.Sequential(
            nn.Conv1d(1, 3, 1),
            nn.BatchNorm1d(3),
            nn.Tanh()
        )
        self.dense_layers.apply(init_weights)

    def forward(self, x):
        batchsize = x.size()[0]
        x = self.dense_layers(x).reshape(batchsize, 1, self.num_points)
        x = self.conv(x)
        return x


'''
apply Conv1d to increase number of points (to 1024)
'''
class PointDecoderPoints(nn.Module):
    def __init__(self, num_points, z_dim):
        super(PointDecoderPoints, self).__init__()
        self.num_points = num_points
        self.z_dim = z_dim
        self.name = f'model_conv1d_points'
        self.conv = nn.Sequential(
            nn.Conv1d(100, 256, 1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Conv1d(256, num_points, 1),
            nn.BatchNorm1d(num_points),
            nn.ReLU(),
        )
        self.dense_layers = nn.Sequential(
            nn.Linear(num_points, num_points*3),
            nn.Dropout(0.05),
            nn.Tanh()
        )
        self.dense_layers.apply(init_weights)

    def forward(self, x):
        batchsize = x.size()[0]
        x = x.reshape(batchsize, self.z_dim, 1)
        x = self.conv(x).reshape(batchsize, self.num_points)
        x = self.dense_layers(x)
        x = x.reshape(batchsize, 3, self.num_points)
        return x

In [ ]:
def train_pcautoencoder(autoencoder, x, loss_func, optimizer):
    '''
    loss function must be chamfer distance
    '''
    optimizer.zero_grad()
    x = x.float().to(device).permute(0, 2, 1)
    output = autoencoder(x)
    dist1, dist2 = loss_func(x, output)

    try:
        # dist2 might be None if x_normals and y_normals (args to loss_func) are None
        loss = (torch.mean(dist1)) + (torch.mean(dist2))
    except:
        loss = (torch.mean(dist1))

    loss.backward()
    optimizer.step()

    return loss.data.item()


def validate_pcautoencoder(autoencoder, x, loss_func):
    '''
    loss function must be chamfer distance
    '''
    with torch.no_grad():
        x = x.float().to(device).permute(0, 2, 1)
        output = autoencoder(x)
        dist1, dist2 = loss_func(x, output)

        try:
            # dist2 might be None if x_normals and y_normals (args to loss_func) are None
            loss = (torch.mean(dist1)) + (torch.mean(dist2))
        except:
            loss = (torch.mean(dist1))

        return loss.data.item()

In [ ]:
def train_with_chamfer_dist(autoencoder, loaders_type, loss_func, optimizer,
                            train_func, validate_func, epochs=100, print_every_e=5, valid_every=5,
                            scheduler=None, summary_writer=None, model_name='model'):
    if loaders_type is DataType.AUG_PRE:
        train_loader, valid_loader = trainloader_pre, validloader_pre
    elif loaders_type is DataType.AUG_DUR:
        train_loader, valid_loader = trainloader_dur, validloader_dur
    else:
        train_loader, valid_loader = trainloader_both, validloader_both

    autoencoder.train()
    for epoch in range(1, epochs+1):
        losses = []
        for x, _ in train_loader:
            loss = train_func(autoencoder, x, loss_func, optimizer)
            losses.append(loss)
        if summary_writer is not None:
            summary_writer.add_scalar(f'{model_name}/train/loss', np.mean(losses), epoch)
        if scheduler:
            scheduler.step()

        if epoch % print_every_e == 0 or epoch == 1:
            print(f'{epoch}:\ttrain loss: {np.mean(losses)}')
        if epoch % valid_every == 0:
            valid_losses = []
            for x, _ in valid_loader:
                valid_loss = validate_func(autoencoder, x, loss_func)
                valid_losses.append(valid_loss)
            if summary_writer is not None:
                summary_writer.add_scalar(f'{model_name}/valid/loss', np.mean(valid_losses), epoch)
            print(f'\tvalidation loss: {np.mean(valid_losses)}')

In [ ]:
def save_model(pca, loaders_type, batchsize=48):
    if loaders_type is DataType.AUG_PRE:
        aug = 'pre'
    elif loaders_type is DataType.AUG_DUR:
        aug = 'dur'
    else:
        aug = 'both'
    torch.save(pca, f'{pca.name}_{aug}_{batchsize}.pt') 

In [ ]:
writer = SummaryWriter()

In [ ]:
pc_autoencoder = PointNetAE(num_points=1024, z_dim=100)
pc_autoencoder.to(device)

optimizer = optim.AdamW(pc_autoencoder.parameters(), lr=0.001, betas=(0.8, 0.8))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)
loss_func = chamfer.chamfer_distance


train_with_chamfer_dist(pc_autoencoder, loaders_type=DataType.AUG_PRE, loss_func=loss_func, optimizer=optimizer,
                        train_func=train_pcautoencoder, validate_func=validate_pcautoencoder,
                        epochs=4000, print_every_e=50, valid_every=100, scheduler=scheduler, summary_writer=writer, 
                        model_name=pc_autoencoder.name)

In [ ]:
# save_model(pc_autoencoder, loaders_type=DataType.AUG_BOTH)
# !cp -r runs drive/MyDrive/

In [ ]:
!ls drive/MyDrive/Thesis/models/dec1\ aug\ pre\ 48

model_dense_pre_48.pt  runs


In [12]:
all_model_paths = [
                   'drive/MyDrive/Thesis/models/dec1 aug pre 48/model_dense_pre_48.pt',
                   'drive/MyDrive/Thesis/models/dec1 aug both 48/model_dense_both_48.pt',
                   'drive/MyDrive/Thesis/models/dec3 aug pre 48/model_conv1d_points_pre_48.pt',
                   'drive/MyDrive/Thesis/models/dec3 aug both 48/model_conv1d_points_both_48.pt'
]

In [ ]:
writer = SummaryWriter()

In [ ]:
# !mv runs drive/MyDrive

In [ ]:
loaders_type = DataType.AUG_BOTH
loss_func = chamfer.chamfer_distance
batch_size = 48
show_orig, show_res = False, False

for model_path in all_model_paths:
    model = torch.load(model_path)
    model.eval()
    model_name = model.name

    loaders_type = DataType.AUG_BOTH if 'both' in model_path else DataType.AUG_PRE
    if loaders_type is DataType.AUG_PRE:
        train_loader, valid_loader = trainloader_pre, validloader_pre
    elif loaders_type is DataType.AUG_DUR:
        train_loader, valid_loader = trainloader_dur, validloader_dur
    else:
        train_loader, valid_loader = trainloader_both, validloader_both

    for valid_num in range(batch_size):
        two_losses = []
        for train in [True, False]:
            local_sample = None
            for sample, _ in train_loader if train else valid_loader:
                sample = sample.permute(0, 2, 1)
                local_sample = sample[valid_num]
                x, y, z = local_sample[:][0], local_sample[:][1], local_sample[:][2]
                if show_orig:
                    pcshow(x, y, z)
                break
            with torch.no_grad():
                samplee = local_sample.unsqueeze(0).float().to(device)
                out = model(samplee)
            first = out[0].detach().cpu()
            x, y, z = first[:][0], first[:][1], first[:][2]
            loss = loss_func(first.unsqueeze(0).float(), local_sample.unsqueeze(0).float())[0].item()
            two_losses.append(loss)
            if show_res:
                pcshow(x, y, z)
        writer.add_scalar(f'{model_name}/{loaders_type}/train', two_losses[0], valid_num)
        writer.add_scalar(f'{model_name}/{loaders_type}/valid', two_losses[1], valid_num)